In [1]:
# upgrade in command line for specific interpreter

import sys
sys.executable
# /Users/lianfeng/Document/species_richness_sdm/venv/bin/pip install --upgrade tensorflow

'/Users/lianfeng/miniconda3/bin/python'

In [2]:
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

Using TensorFlow backend.


In [12]:
# 
exec('from __future__ import absolute_import, division, print_function, unicode_literals')

import numpy as np
import datetime
import pathlib
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Activation, Dropout

In [5]:
# Use some functions from tensorflow_docs
!pip install -q git+https://github.com/tensorflow/docs
    
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from tensorboard.plugins.hparams import api as hp

In [15]:
localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))

pergrid_base = """select distinct * from predictor.pergrid_base"""
pergrid_base_df = pd.read_sql(pergrid_base, engine)
pergrid_base_df_na_remove = pergrid_base_df.dropna()

pergrid_expanded_encoded=pd.get_dummies(pergrid_base_df_na_remove, columns=["shg"])

pred_var= ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
   'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt',
   'mtwq','shg_A', 'shg_A/D', 'shg_B', 'shg_B/D', 'shg_C', 'shg_C/D', 'shg_D']

grid_id = pergrid_base_df_na_remove['grid_id']
y = np.asarray(pergrid_base_df_na_remove['tsr'])
x = np.asarray(pergrid_expanded_encoded[pred_var])

In [7]:
def data():
    localhost = {'user': 'postgres', 'password': 'postgres', 'host': 'localhost', 'port': 5432, 'db': 'fiadb'}
    params = 'postgresql://{0}:{1}@{2}:{3}/{4}'
    engine = create_engine(params.format(localhost['user'], localhost['password'], localhost['host'], localhost['port'], localhost['db']))
    
    pergrid_base = """select distinct * from predictor.pergrid_base"""
    pergrid_base_df = pd.read_sql(pergrid_base, engine)
    pergrid_base_df_na_remove = pergrid_base_df.dropna()

    pergrid_expanded_encoded=pd.get_dummies(pergrid_base_df_na_remove, columns=["shg"])

    pred_var= ['aet', 'ai', 'art', 'ewd', 'fa', 'map', 'mat', 'mpdq',
       'mtcq', 'pet', 'psn', 'ra', 'rmap', 'rmat', 'tsn', 'mfdf', 'alt',
       'mtwq','shg_A', 'shg_A/D', 'shg_B', 'shg_B/D', 'shg_C', 'shg_C/D', 'shg_D']

    grid_id = pergrid_base_df_na_remove['grid_id']
    y = pergrid_base_df_na_remove['tsr']
    x = pergrid_expanded_encoded[pred_var]

    # normalize features 
    x_stats = x.describe()
    x_stats = x_stats.transpose()
    x = (x - x_stats['mean']) / x_stats['std']

    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=12345)
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=12345)
    # return x_train.to_numpy(), x_val.to_numpy(), x_test.to_numpy(), y_train.to_numpy(), y_val.to_numpy(), y_test.to_numpy()
    return np.asarray(x_train), np.asarray(x_val), np.asarray(x_test), np.asarray(y_train), np.asarray(y_val), np.asarray(y_test)

In [22]:
def create_model(x_train, y_train, x_val, y_val):

    model = keras.Sequential()
    model.add(layers.Dense({{choice([128, 256, 512, 1024])}}, input_shape=(x_train.shape[1],)))  # tune # of units
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))  # tune the activation function
    model.add(Dropout({{uniform(0, 1)}}))  # tune dropout rate

    model.add(Dense({{choice([128, 256, 512, 1024])}}))
    model.add(Activation({{choice(['relu', 'sigmoid'])}}))
    model.add(Dropout({{uniform(0, 1)}}))

    # tune # of layers
    if {{choice(['two', 'three'])}} == 'three':
        model.add(Dense({{choice([128, 256, 512, 1024])}}))
        model.add(Activation({{choice(['relu', 'sigmoid'])}}))
        model.add(Dropout({{uniform(0, 1)}}))

    model.add(Dense(1))
    model.add(Activation('linear'))

    # tune the optimizer and the learning rate
    adam = keras.optimizers.Adam(lr={{choice([10 ** -3, 10 ** -2, 10 ** -1])}})
    rmsprop = keras.optimizers.RMSprop(lr={{choice([10 ** -3, 10 ** -2, 10 ** -1])}})
    sgd = keras.optimizers.SGD(lr={{choice([10 ** -3, 10 ** -2, 10 ** -1])}})

    choiceval = {{choice(['adam', 'sgd', 'rmsprop'])}}
    if choiceval == 'adam':
        optim = adam
    elif choiceval == 'rmsprop':
        optim = rmsprop
    else:
        optim = sgd

    model.compile(loss='mae', metrics=['mean_squared_error'], optimizer=optim)
    result=model.fit(x_train, y_train,
              batch_size={{choice([128, 256, 512])}},
              nb_epoch=1000,
              verbose=2,
              validation_data=(x_val, y_val))
    
    mae, mse = model.evaluate(x_val, y_val, verbose=0)
    print('MSE:', mse)
    return {'loss': mae, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=create_model,
                                      data=data,
                                      algo=tpe.suggest,
                                      max_evals=30,
                                      trials=Trials(),
                                      notebook_name='M5_FNN')

>>> Imports:
#coding=utf-8

try:
    import sys
except:
    pass

try:
    from hyperopt import Trials, STATUS_OK, tpe
except:
    pass

try:
    from hyperas import optim
except:
    pass

try:
    from hyperas.distributions import choice, uniform
except:
    pass

try:
    import numpy as np
except:
    pass

try:
    import datetime
except:
    pass

try:
    import pathlib
except:
    pass

try:
    from sqlalchemy import create_engine
except:
    pass

try:
    import matplotlib.pyplot as plt
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import seaborn as sns
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn import preprocessing
except:
    pass

try:
    from tensorflow import keras
except:
    pass

try:
    from tensorflow.keras import layers
except:
    pass

try:
    from tensorflow.keras.layers import Dense, Activation, Dropout
except:
    pass

try:
    import tensorflow_docs a

8641/8641 - 1s - loss: 4.0721 - mean_squared_error: 32.3144 - val_loss: 3.4421 - val_mean_squared_error: 23.0322

Epoch 15/1000                                         
8641/8641 - 1s - loss: 4.0725 - mean_squared_error: 31.6681 - val_loss: 3.4407 - val_mean_squared_error: 23.1883

Epoch 16/1000                                         
8641/8641 - 1s - loss: 3.9809 - mean_squared_error: 31.2724 - val_loss: 4.0094 - val_mean_squared_error: 29.3773

Epoch 17/1000                                         
8641/8641 - 1s - loss: 4.0289 - mean_squared_error: 31.0722 - val_loss: 3.4636 - val_mean_squared_error: 23.3350

Epoch 18/1000                                         
8641/8641 - 1s - loss: 3.9816 - mean_squared_error: 30.6452 - val_loss: 3.7091 - val_mean_squared_error: 25.0124

Epoch 19/1000                                         
8641/8641 - 1s - loss: 3.9938 - mean_squared_error: 30.9079 - val_loss: 3.3970 - val_mean_squared_error: 22.3332

Epoch 20/1000                            

Epoch 63/1000                                         
8641/8641 - 1s - loss: 3.4666 - mean_squared_error: 23.9728 - val_loss: 3.1980 - val_mean_squared_error: 19.9115

Epoch 64/1000                                         
8641/8641 - 1s - loss: 3.5478 - mean_squared_error: 24.9782 - val_loss: 3.5259 - val_mean_squared_error: 24.6312

Epoch 65/1000                                         
8641/8641 - 1s - loss: 3.4219 - mean_squared_error: 23.1891 - val_loss: 3.5174 - val_mean_squared_error: 24.3324

Epoch 66/1000                                         
8641/8641 - 1s - loss: 3.4623 - mean_squared_error: 23.6548 - val_loss: 3.3773 - val_mean_squared_error: 22.8587

Epoch 67/1000                                         
8641/8641 - 1s - loss: 3.5166 - mean_squared_error: 24.9374 - val_loss: 3.3495 - val_mean_squared_error: 21.9806

Epoch 68/1000                                         
8641/8641 - 1s - loss: 3.4388 - mean_squared_error: 23.6718 - val_loss: 3.6596 - val_mean_squared_er

8641/8641 - 1s - loss: 3.3564 - mean_squared_error: 22.5834 - val_loss: 3.1428 - val_mean_squared_error: 19.3327

Epoch 112/1000                                        
8641/8641 - 1s - loss: 3.3598 - mean_squared_error: 22.7388 - val_loss: 3.2851 - val_mean_squared_error: 21.2757

Epoch 113/1000                                        
8641/8641 - 1s - loss: 3.3574 - mean_squared_error: 22.5040 - val_loss: 3.3468 - val_mean_squared_error: 21.7896

Epoch 114/1000                                        
8641/8641 - 1s - loss: 3.3427 - mean_squared_error: 22.2784 - val_loss: 3.9531 - val_mean_squared_error: 30.2563

Epoch 115/1000                                        
8641/8641 - 1s - loss: 3.3413 - mean_squared_error: 22.5829 - val_loss: 3.2779 - val_mean_squared_error: 21.3058

Epoch 116/1000                                        
8641/8641 - 1s - loss: 3.3239 - mean_squared_error: 22.1373 - val_loss: 3.6322 - val_mean_squared_error: 25.7035

Epoch 117/1000                           

Epoch 160/1000                                        
8641/8641 - 1s - loss: 3.2435 - mean_squared_error: 21.4715 - val_loss: 3.3096 - val_mean_squared_error: 21.9772

Epoch 161/1000                                        
8641/8641 - 1s - loss: 3.1904 - mean_squared_error: 20.5638 - val_loss: 3.2733 - val_mean_squared_error: 21.3338

Epoch 162/1000                                        
8641/8641 - 1s - loss: 3.2731 - mean_squared_error: 21.6429 - val_loss: 3.6273 - val_mean_squared_error: 26.4775

Epoch 163/1000                                        
8641/8641 - 1s - loss: 3.1864 - mean_squared_error: 20.6632 - val_loss: 3.2563 - val_mean_squared_error: 20.9525

Epoch 164/1000                                        
8641/8641 - 1s - loss: 3.2925 - mean_squared_error: 21.8001 - val_loss: 3.7356 - val_mean_squared_error: 27.0235

Epoch 165/1000                                        
8641/8641 - 1s - loss: 3.2201 - mean_squared_error: 21.0723 - val_loss: 3.2606 - val_mean_squared_er

...

Epoch 882/1000                                                                       
8641/8641 - 1s - loss: 6.8436 - mean_squared_error: 91.2231 - val_loss: 6.9187 - val_mean_squared_error: 87.6899

Epoch 883/1000                                                                       
8641/8641 - 1s - loss: 6.7701 - mean_squared_error: 86.9601 - val_loss: 6.6611 - val_mean_squared_error: 88.2602

Epoch 884/1000                                                                       
8641/8641 - 1s - loss: 6.7606 - mean_squared_error: 88.2347 - val_loss: 6.5415 - val_mean_squared_error: 79.7112

Epoch 885/1000                                                                       
8641/8641 - 1s - loss: 6.8068 - mean_squared_error: 87.2265 - val_loss: 6.9131 - val_mean_squared_error: 87.7288

Epoch 886/1000                                                                       
8641/8641 - 1s - loss: 6.7909 - mean_squared_error: 87.3759 - val_loss: 6.5395 - val_mean_squared_error: 78.3234



Epoch 923/1000                                                                       
8641/8641 - 2s - loss: 6.7434 - mean_squared_error: 83.6924 - val_loss: 6.6352 - val_mean_squared_error: 82.7951

Epoch 924/1000                                                                       
8641/8641 - 1s - loss: 6.6843 - mean_squared_error: 84.7278 - val_loss: 6.6417 - val_mean_squared_error: 81.9310

Epoch 925/1000                                                                       
8641/8641 - 1s - loss: 6.7134 - mean_squared_error: 83.0644 - val_loss: 6.6897 - val_mean_squared_error: 83.2842

Epoch 926/1000                                                                       
8641/8641 - 2s - loss: 6.7723 - mean_squared_error: 86.2860 - val_loss: 6.5321 - val_mean_squared_error: 80.2444

Epoch 927/1000                                                                       
8641/8641 - 1s - loss: 6.6989 - mean_squared_error: 82.5236 - val_loss: 6.6596 - val_mean_squared_error: 84.2615



Epoch 964/1000                                                                       
8641/8641 - 1s - loss: 6.7591 - mean_squared_error: 84.4750 - val_loss: 6.8364 - val_mean_squared_error: 87.5630

Epoch 965/1000                                                                       
8641/8641 - 1s - loss: 6.7659 - mean_squared_error: 90.5513 - val_loss: 6.7604 - val_mean_squared_error: 83.8052

Epoch 966/1000                                                                       
8641/8641 - 1s - loss: 6.7728 - mean_squared_error: 87.2968 - val_loss: 6.6214 - val_mean_squared_error: 81.4144

Epoch 967/1000                                                                       
8641/8641 - 1s - loss: 6.6722 - mean_squared_error: 84.7630 - val_loss: 6.6740 - val_mean_squared_error: 83.2919

Epoch 968/1000                                                                       
8641/8641 - 1s - loss: 6.7286 - mean_squared_error: 82.6374 - val_loss: 6.4563 - val_mean_squared_error: 77.8281



Train on 8641 samples, validate on 2161 samples                                      
Epoch 1/1000                                                                         
8641/8641 - 2s - loss: 10.8004 - mean_squared_error: 198.6636 - val_loss: 8.4461 - val_mean_squared_error: 125.0656

Epoch 2/1000                                                                         
8641/8641 - 1s - loss: 7.1870 - mean_squared_error: 93.6811 - val_loss: 6.1442 - val_mean_squared_error: 70.4896

Epoch 3/1000                                                                         
8641/8641 - 1s - loss: 5.6740 - mean_squared_error: 59.8017 - val_loss: 5.0934 - val_mean_squared_error: 51.2021

Epoch 4/1000                                                                         
8641/8641 - 1s - loss: 4.9988 - mean_squared_error: 47.3155 - val_loss: 4.6511 - val_mean_squared_error: 41.6506

Epoch 5/1000                                                                         
8641/8641 - 1s - loss: 4.

8641/8641 - 1s - loss: 3.5316 - mean_squared_error: 24.6861 - val_loss: 3.4711 - val_mean_squared_error: 23.8085

Epoch 42/1000                                                                        
8641/8641 - 1s - loss: 3.5388 - mean_squared_error: 24.9336 - val_loss: 3.4593 - val_mean_squared_error: 23.6494

Epoch 43/1000                                                                        
8641/8641 - 1s - loss: 3.5264 - mean_squared_error: 24.6067 - val_loss: 3.4658 - val_mean_squared_error: 23.8138

Epoch 44/1000                                                                        
8641/8641 - 1s - loss: 3.5238 - mean_squared_error: 24.6966 - val_loss: 3.4917 - val_mean_squared_error: 23.9365

Epoch 45/1000                                                                        
8641/8641 - 1s - loss: 3.5203 - mean_squared_error: 24.5373 - val_loss: 3.4708 - val_mean_squared_error: 23.5915

Epoch 46/1000                                                                        


In [10]:
print(best_run)

{'Activation': 0, 'Activation_1': 1, 'Activation_2': 1, 'Dense': 3, 'Dense_1': 2, 'Dense_2': 3, 'Dropout': 0.5477841273226159, 'Dropout_1': 0.7687479985485637, 'Dropout_2': 1, 'Dropout_3': 0.23792781723659964, 'batch_size': 0, 'choiceval': 0, 'lr': 0, 'lr_1': 2, 'lr_2': 1}


In [13]:
X_train, X_val, X_test, Y_train, Y_val, Y_test = data()

In [14]:
best_model.evaluate(X_test, Y_test, verbose=0)

[3.230249714824898, 20.375944]

### Make predictions

Finally, predict TSR values using data in the testing set:

In [16]:
Y_test_pred = best_model.predict(X_test).flatten()

It looks like our model predicts reasonably well. Let's take a look at the error distribution.

In [17]:
error = Y_test - Y_test_pred
plt.hist(error, bins = 25)
plt.xlabel("Prediction Error [TSR]")
_ = plt.ylabel("Count")

### Make TSR predictions using the full dataset

In [19]:
# test_predictions = model.predict(normed_test_data).flatten()
y_pred = best_model.predict(x).flatten()

### Ingest TSR predictions to PG

In [20]:
pergrid_all_predicted = pd.DataFrame(
    {'grid_id': grid_id,
     'tsr': y,
     'tsr_predicted': y_pred})

In [21]:
pergrid_all_predicted.head()

grid_id  tsr  tsr_predicted
1      111  2.0       8.390343
4      338  1.0       8.116247
5      339  2.0       8.116247
6      340  2.0       8.116247
7      342  2.0       8.116247

In [190]:
pergrid_all_predicted.to_sql(name='m5_base_fnn', con=engine, schema='predictor', if_exists='replace', index=False)

In [191]:
update_geom = """
alter table predictor.m5_base_fnn add column if not exists wkb_geometry geometry(Polygon,4269);
update predictor.m5_base_fnn A SET wkb_geometry = B.wkb_geometry
FROM predictor.pergrid_base B
WHERE A.grid_id = B.grid_id
"""

In [192]:
connection = engine.connect()
connection.execute(update_geom)

In [193]:
update_residual = """
alter table predictor.m5_base_fnn add column residual double precision;
update predictor.m5_base_fnn set residual = (tsr_predicted-tsr);
"""

In [194]:
connection = engine.connect()
connection.execute(update_residual)